<a href="https://colab.research.google.com/github/FilipeMell0/AnimaCursoDeExtensaoPythonPandas2023-1/blob/main/Mais_exemplos_em_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Segundo exemplo de manipulação de dados usando Python Pandas 

Veremos inserção de dados, remoção de colunas, merge, gráficos

Autor: Filipe Mello e Poyatão da massa.

Data: 9 e 10 de fevereiro de 2023

In [ ]:
#Importando biblioteca pandas no Pyhton
import pandas as pd

combustiveis_df = pd.read_excel("ca-2021-02.xlsx")
display(combustiveis_df.head())

In [ ]:
#Insersão simples de dados 
combustiveis_df['Ativo'] = True 
display(combustiveis_df.head())

In [ ]:
#Criar uma coluna "Obs" que tenha nela escrito "MELHOR CIDADE" quando a coluna municipio foi igual a PORTO ALEGRE
combustiveis_df['Obs'] = ["MELHOR CIDADE" if municipio == 'PORTO ALEGRE' else None for municipio in combustiveis_df['Municipio']]
display(combustiveis_df.loc[combustiveis_df['Municipio'].isin(['PORTO ALEGRE','ALVORADA', 'GRAMADO', 'CANELA']), ['Municipio', 'Obs']])

In [ ]:
#Como preencher uma coluna 'Valor de venda - Status' que verifica o seguinte:
#Se o valor de venda foi maior que R$6,5 ele diz que está caro, caso contrário, dirá que está barato
import numpy as np

combustiveis_df['Status do valor de venda'] = np.where(combustiveis_df['Valor de Venda'] > 6.5, 'Caro','Barato')
display(combustiveis_df[['Revenda','Valor de Venda','Status do valor de venda']])

In [ ]:
#Calcular postos de gasolina por habitante temos na amostragem de combustiveis nov/2021

num_habitantes_df = pd.read_csv("ibge_num_habitantes_estimado.csv", sep = ";")
num_habitantes_df.rename(columns={"Estado":"Estado - Sigla"}, inplace = True)
display(num_habitantes_df)

In [ ]:
#Fazer um MERGE dos dois dataframes
colunas = ['Municipio', 'Estado - Sigla']
merge_df = combustiveis_df.merge(num_habitantes_df, how="inner", on=colunas)
display(merge_df)
print(merge_df.info())

In [ ]:
#Destruir coluna completamente vazia (todas as linhas são nulas)
merge_df.dropna(axis='columns', inplace=True)
print(merge_df.info())

In [ ]:
colunas=['Regiao - Sigla', 'Nome da Rua', 'Numero Rua', 
         'Bairro', 'Cep', 'Produto', 'Data da Coleta', 'Valor de Venda',
         'Unidade de Medida', 'Bandeira', 'Ativo', 'Status do valor de venda']
merge_df.drop(labels=colunas, axis=1, inplace=True)
print(merge_df.info())

In [ ]:
# Remover a linhas duplicadas
merge_df.drop_duplicates(inplace=True)
display(merge_df.head(100))

In [ ]:
#Agrupar e contar quantos postos tem na cidade..
postos_por_municipio_df = merge_df.groupby(by=['Estado - Sigla', 'Municipio', 'NumHabitantes2021']).count()
postos_por_municipio_df.drop('CNPJ da Revenda', axis=1, inplace=True)
postos_por_municipio_df.rename(columns={"Revenda": "Número de Postos"}, inplace=True)
display(postos_por_municipio_df)

In [ ]:
#Agrupar e contar quantos postos tem na cidade..
postos_por_municipio_df = merge_df.groupby(by=['Estado - Sigla', 'Municipio', 'NumHabitantes2021']).count()
postos_por_municipio_df.reset_index(inplace=True)
#display(postos_por_municipio_df.info())
postos_por_municipio_df.drop('CNPJ da Revenda', axis=1, inplace=True)
postos_por_municipio_df.rename(columns={"Revenda": "NumPostos"}, inplace=True)

postos_por_municipio_df['NumHabitantesPorPosto'] = postos_por_municipio_df['NumHabitantes2021'] / postos_por_municipio_df['NumPostos']
#display(postos_por_municipio_df.info())
display(postos_por_municipio_df)

In [ ]:
#Vamos brincar de gráficos!
import matplotlib.pyplot as plt

plt.hist(combustiveis_df["Valor de Venda"])

#Vamos colocar um título no gráfico
plt.title("Preço dos combustíveis - Nov 2021")

#Rótulo horizontal 
plt.xlabel("Preço (em reais)")
plt.ylabel("Quantidade de coletas")

#Traça a linha vermelha tracejada com preço médio
plt.axvline(combustiveis_df["Valor de Venda"].mean(), color='red',linestyle='dashed',linewidth=5)

#"Plota" o gráfico
plt.show()

In [ ]:
#Visualização do consumo médio

c_mean = combustiveis_df['Valor de Venda'].groupby(by=combustiveis_df['Produto']).mean()

display(c_mean)

In [ ]:
import seaborn as sns

#Definindo a área do gráfico
plt.figure(figsize=(7,5))

#Plotar o gráfico
c_mean.plot(
    kind="barh",
    title="Média de preço dos combustíveis",
    xlabel="Tipo de Combustível",
    ylabel="Preço reais/litro",
    color="red",
    alpha=0.3
)

#Grid
plt.grid()

#Remover as linhas superiores e laterais do gráfico
sns.despine()

#Exibe
plt.show()


In [ ]:
#c_mean.reset_index(inplace=True)
display(c_mean)

excel = "por_litro.xlsx"
c_mean.to_excel(excel, "Sumário")

In [52]:
#Vamos personalizar a planilha do excel
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Color, Alignment

#Vai abrir o excel no Openyxl 
wb = load_workbook(excel) #wb = workbook


#Pegar a planilha certa... usando o Sheet name (nome da planilha)
ws = wb['Sumário'] #Work Sheet -> planilha atual, ativa, de trabalho

#Vamos pintar o cabeçalho da tabela de "cinzinha"
cinzinha = PatternFill("solid", fgColor = "cccccc")
coords = ['A1','B1']
for cord in coords:
  ws[cord].fill = cinzinha

#Onde o preço do combustível for maior que 6,5 reais pinte de vermelho e deixa negrito
MAX_ROW = ws.max_row
num_linha = 2
while (num_linha <= MAX_ROW):
  coord = 'B'+str(num_linha) #coord="B{0}".format(num_linha)
  if ws[coord].value >= 6.5:
    ws[coord].font = Font(bold=True, color="FF0000")
  num_linha = num_linha + 1

#Salvar o excel 
wb.save(excel)
